# Pylops - Solvers with distributed operator

In this notebook we investigate the use of scipy solvers with distributed operators. 
We will also create a simple CG solver to compare the results and performance

In [100]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import skfmm
import dask.array as da
import pylops
import pylops_distributed

from scipy.sparse.linalg.interface import MatrixLinearOperator, aslinearoperator 
from scipy.sparse import csr_matrix, vstack
from scipy.linalg import lstsq, solve
from scipy.sparse.linalg import LinearOperator, cg, lsqr, gmres

from scipy.signal import convolve, filtfilt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
client = pylops_distributed.utils.backend.dask()
client

Client Scheduler: tcp://127.0.0.1:60593 Dashboard: http://127.0.0.1:60594/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


## CG solver

In [168]:
def conjgrad(A, b, x0, niter):
    'Conjugate-gradient algorithm for minimizing |A x - dat|^2'
    x = x0
    r = b-A._matvec(x).ravel()
    d = r.copy()
    k = r.dot(r)
    k0 = k
    for iter in range(niter):
        a = r.dot(r) / d.dot(A._matvec(d).ravel())
        x = x + a*d
        rnew = r - a*A._matvec(d).ravel()
        k = rnew.dot(rnew)
        b = rnew.dot(rnew) / r.dot(r)
        d = rnew + b*d;
        r = rnew.copy()
    return x

In [170]:
n = 10
x = np.ones(n)

A = np.random.randn(n, n)
A = np.dot(A.T, A)
print('eigs', np.linalg.eig(A)[0])

#A = MatrixLinearOperator(A)
Aop = aslinearoperator(A)

y = Aop.matvec(x)
xinv = conjgrad(Aop, y, np.zeros_like(x), n)
xinv_sp = cg(Aop, y, maxiter=n)[0]
print(xinv)
print(xinv_sp)

eigs [32.91513571 25.26839471 20.05106997 13.7007825  10.5851348   8.76075269
  5.66108672  0.09491681  0.54733873  2.87850113]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [171]:
Ada = da.from_array(A, chunks=(n//2, n//2))
Aop = MatrixLinearOperator(Ada)
y = Aop._matvec(da.ones(n, chunks=(n//2,))).compute().ravel()

xinv = conjgrad(Aop, y, np.zeros_like(x), n)
xinv_sp = gmres(Aop, y, maxiter=n)[0]
print(xinv.compute())
print(xinv_sp)

dask.array<add, shape=(10,), dtype=float64, chunksize=(5,)>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [175]:
%timeit -n 3 -r 1 gmres(Aop, y, maxiter=n)[0]
%timeit -n 3 -r 1 conjgrad(Aop, y, np.zeros_like(x), n)

4.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:61479, threads: 1>>
Traceback (most recent call last):
  File "/Users/matteoravasi/anaconda/envs/pylops-distributed/lib/python3.7/site-packages/psutil/_common.py", line 342, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/matteoravasi/anaconda/envs/pylops-distributed/lib/python3.7/site-packages/psutil/_common.py", line 342, in wrapper
    ret = self._cache[fun]
AttributeError: _cache

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/matteoravasi/anaconda/envs/pylops-distributed/lib/python3.7/site-packages/psutil/_psosx.py", line 359, in catch_zombie
    yield
  File "/Users/matteoravasi/anaconda/envs/pylops-distributed/lib/python3.7/site-p

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker


In [ ]:
gmres(Aop, y, maxiter=n)[0]

**Note:** in order to allow lazy evaluation of mat-vec products in a solver (leading to a dask array ``xinv``), it is required to use ``_matvec`` and  ``_rmatvec`` instead of 
``matvec`` and  ``rmatvec``. However scipy solvers use the latter... 

Moreover the graph creation when allowing lazy evaluation seem to be the bottleneck
what is the best way forward?